This notebook is a layout for OLS regression models run in the state of Illinois by census tract. 

The model accounts for the total count of vision facilities in comparison to the socioeconomic variables (percent white population, median household income, and percent with a bachelors degree or higher).

Multiple models were not ran through this notebook due to the realization that different vision types would likely need to be specified and ran through seperate regression models, but this notebook still lies a great framework for how future OLS models should be set up in python, including how to conduct transformations on specific variables in order to satisfy the assumptions of OLS regression. 

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
#import geoplot as gplt
import contextily
from shapely.geometry import Point
from shapely import wkt
pd.set_option('display.max_columns', None)
import pyarrow as pyarrow
#import fastparquet as fastparquet
#import datasets
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
vision=pd.read_parquet("../Vision Work/vision_providers_minimal.parquet")
#county = gpd.read_file("/srv/data/my_shared_data_folder/pb-data/county.gpkg")
#zip_to_county = gpd.read_file("../data/ZIP_COUNTY_032023 (1).csv")
ILcentroids = pd.read_csv("../Geospatial/ILCensusCentroids.csv")
# link to ill censustracts centroids csv: https://www2.census.gov/geo/docs/reference/cenpop2020/tract/CenPop2020_Mean_TR17.txt

In [2]:
# %load ../standard_import.txt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import seaborn as sns

from sklearn.preprocessing import scale
import sklearn.linear_model as skl_lm
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import statsmodels.formula.api as smf

%matplotlib inline
plt.style.use('seaborn-white')

/tmp/ipykernel_2641389/1698868546.py:15: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-white')


In [4]:
#subsetting vision provider data based on classification (ie optometrist/ophthamologist) and entity type
vision2 = vision[vision["Entity Type Code"] == 1]
vision2['Optometry'] = vision2['Taxonomy'].apply(lambda x: any(code.startswith('152') for code in x.split('|')))
vision2['Ophthalmology'] = vision2['Taxonomy'].apply(lambda x: any(code.startswith('207') for code in x.split('|')))
vision2['Others'] = vision2['Taxonomy'].apply(lambda x: any(code.startswith('156') for code in x.split('|')))

# creating geodataframes and subsetting data for just illinois
ILcentroids = gpd.GeoDataFrame(
   ILcentroids, geometry=gpd.points_from_xy(ILcentroids.LONGITUDE, ILcentroids.LATITUDE),
    crs="EPSG:4326")

vision3 = gpd.GeoDataFrame(
   vision2, geometry=gpd.points_from_xy(vision2.Longitude, vision2.Latitude),
    crs="EPSG:4326")
vision3 = vision3[vision3["Provider Business Mailing Address State Name"] == 'IL']

#changing crs to projected crs
ILcentroidsp = ILcentroids.to_crs("ESRI:102008")
vision3p = vision3.to_crs("ESRI:102008")

/tmp/ipykernel_2641389/4025650584.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vision2['Optometry'] = vision2['Taxonomy'].apply(lambda x: any(code.startswith('152') for code in x.split('|')))
/tmp/ipykernel_2641389/4025650584.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vision2['Ophthalmology'] = vision2['Taxonomy'].apply(lambda x: any(code.startswith('207') for code in x.split('|')))
/tmp/ipykernel_2641389/4025650584.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [6]:
# code demonstrating how to merge dataframes based on nearest
ILmerged = ILcentroidsp.sjoin_nearest(vision3p, how='inner', lsuffix='left', rsuffix='right', distance_col="Distances")
ILdistances = ILmerged.drop_duplicates(subset=['Distances'])
ILdistances

,STATEFP,COUNTYFP,TRACTCE,POPULATION,LATITUDE,LONGITUDE,geometry,index_right,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,Provider Other First Name,Provider Other Middle Name,Provider Other Name Prefix Text,Provider Other Name Suffix Text,Provider Other Credential Text,Provider Other Last Name Type Code,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code,Provider Business Mailing Address Country Code (If outside U.S.),Provider Gender Code,Authorized Official Last Name,Authorized Official First Name,Authorized Official Middle Name,Authorized Official Title or Position,Authorized Official Telephone Number,Certification Date,Clean Zip,Full Address,Latitude,Longitude,Taxonomy,Optometry,Ophthalmology,Others,Distances
0,17,1,100,4644,39.943585,-91.362273,POINT (372954.658 2470.038),6717,1629155379,1,NaN,None,None,SPOONER,BRIAN,S,None,None,O.D.,None,NaN,None,None,None,None,None,None,NaN,3400 BROADWAY,QUINCY MALL,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,3400 BROADWAY QUINCY MALL QUINCY IL 62301,39.936666,-91.363201,152W00000X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,True,False,False,819.484510
1,17,1,201,2067,39.953942,-91.387764,POINT (370846.943 3590.241),15330,1578752853,1,NaN,None,None,DOUGHERTY,CHRIS,None,MR.,None,None,None,NaN,None,None,None,None,None,None,NaN,1 STATE AND 8TH PLZ,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,1 STATE AND 8TH PLZ QUINCY IL 62301,39.942065,-91.395169,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,1522.315718
2,17,1,202,2870,39.941336,-91.385826,POINT (371074.990 2112.646),15330,1578752853,1,NaN,None,None,DOUGHERTY,CHRIS,None,MR.,None,None,None,NaN,None,None,None,None,None,None,NaN,1 STATE AND 8TH PLZ,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,1 STATE AND 8TH PLZ QUINCY IL 62301,39.942065,-91.395169,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,756.601345
3,17,1,400,3793,39.948935,-91.406851,POINT (369342.060 2925.819),15330,1578752853,1,NaN,None,None,DOUGHERTY,CHRIS,None,MR.,None,None,None,NaN,None,None,None,None,None,None,NaN,1 STATE AND 8TH PLZ,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,1 STATE AND 8TH PLZ QUINCY IL 62301,39.942065,-91.395169,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,1240.863085
4,17,1,500,1719,39.944326,-91.398339,POINT (370052.365 2416.000),15330,1578752853,1,NaN,None,None,DOUGHERTY,CHRIS,None,MR.,None,None,None,NaN,None,None,None,None,None,None,NaN,1 STATE AND 8TH PLZ,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,1 STATE AND 8TH PLZ QUINCY IL 62301,39.942065,-91.395169,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,368.957210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3248,17,201,4002,6613,42.456448,-89.055590,POINT (536459.032 309725.343),117529,1407386667,1,NaN,None,None,HANSEN,RYAN,None,None,None,OD,None,NaN,None,None,None,None,None,None,NaN,4775 E ROCKTON RD UNIT A,None,ROSCOE,IL,61073,US,M,None,None,None,None,NaN,07/19/2021,61073,4775 E ROCKTON RD UNIT A ROSCOE IL 61073,42.455967,-89.024281,152W00000X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,True,False,False,2421.493091
3249,17,201,4003,2102,42.488844,-89.061292,POINT (535739.372 313511.231),117529,1407386667,1,NaN,None,None,HANSEN,RYAN,None,None,None,OD,None,NaN,None,None,None,None,None,None,NaN,4775 E ROCKTON RD UNIT A,None,ROSCOE,IL,61073,US,M,None,None,None,None,NaN,07/19

In [7]:
#Read in convert file to create shared TRACTCE column
convert = pd.read_csv("../IL Work/ILTractToGeoID.csv")

In [8]:
#Drop duplicates
convert = convert.drop_duplicates()

In [15]:
#Merge convert and the distances from the nearest eyecare facility dataframes
mergedTract=pd.merge(ILdistances, convert, how='inner', on= "TRACTCE")

In [16]:
#Drop duplicate values within the GEOID columns
mergedTract = mergedTract.drop_duplicates(subset=['GEOID'])

In [17]:
#Reset the index of the merged Tract df
mergedTract= mergedTract.reset_index()

In [19]:
#Read in the socioeconomic characteristics
socio= pd.read_csv("../IL Work/socio_econ_factor_with_counts_IL_tracts.csv", low_memory=False)

In [21]:
#Set GEOID column of socio to a string datatype
socio['GEOID']=socio['GEOID'].astype(str)

In [22]:
#Set GEOID column of mergedTract to a string datatype
mergedTract['GEOID']=mergedTract['GEOID'].astype(str)

In [23]:
#Merge the socio and mergedTract dataframes
socio_econ_factors_distance=pd.merge(socio, mergedTract, how= 'left', on= "GEOID")

In [26]:
#Conduct log transform on distances from nearest facility to create OLS model that satisfies assumptions of OLS and create as new column
socio_econ_factors_distance = socio_econ_factors_distance.assign(DistanceLog = np.log(socio_econ_factors_distance['Distances']))

In [28]:
#Conduct log transform on white population percentage to create OLS model that satisfies assumptions of OLS and create as new column
socio_econ_factors_distance = socio_econ_factors_distance.assign(WhitePercLog = np.log(socio_econ_factors_distance['WhitePerc']))

In [30]:
#Conduct log transform on median household income to create OLS model that satisfies assumptions of OLS and create as new column
socio_econ_factors_distance = socio_econ_factors_distance.assign(MedianIncomeLog = np.log(socio_econ_factors_distance['Median_household_income']))

/opt/tljh/user/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [31]:
#Conduct log transform on percentage of population with a bachelors degree or higher to create OLS model that satisfies assumptions of OLS and create as new column
socio_econ_factors_distance = socio_econ_factors_distance.assign(BachelorLog = np.log(socio_econ_factors_distance['Bach_or_higher_perc']))

/opt/tljh/user/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [34]:
#Conduct log transform on total count of facilities per census tract to create OLS model that satisfies assumptions of OLS and create as new column
socio_econ_factors_distance = socio_econ_factors_distance.assign(CountLog = np.log(socio_econ_factors_distance['Total count']+1))

In [36]:
#Turn new df that includes log transforms into CSV file
socio_econ_factors_distance.to_csv('socio_econ_factors_distance_f.csv', index = False)

In [37]:
# Use these values to set up the grid for plotting.
socio_econ_factors_distance[['WhitePerc', 'Median_household_income', 'Bach_or_higher_perc']].describe()

,WhitePerc,Median_household_income,Bach_or_higher_perc
count,3265.000000,3265.000000,3265.000000
mean,60.081268,75695.633384,34.184839
std,29.748974,37798.706120,21.833770
min,0.000000,0.000000,0.000000
25%,38.930000,51066.000000,16.900000
50%,69.110000,68347.000000,28.400000
75%,84.790000,93155.000000,47.300000
max,97.720000,250000.000000,95.200000


In [38]:
#Set log of count as dependent variables
y2 = socio_econ_factors_distance['CountLog']

In [39]:
#Set demographic info as independent variables
x = socio_econ_factors_distance[['WhitePerc', 'Median_household_income', 'Bach_or_higher_perc']]

In [40]:
x = sm.add_constant(x)

In [41]:
#Run regression OLS model utilizing log count transform
model2 = sm.OLS(y2, x).fit()

In [42]:
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:               CountLog   R-squared:                       0.170
Model:                            OLS   Adj. R-squared:                  0.170
Method:                 Least Squares   F-statistic:                     223.2
Date:                Wed, 26 Jul 2023   Prob (F-statistic):          1.13e-131
Time:                        18:56:03   Log-Likelihood:                -5173.6
No. Observations:                3265   AIC:                         1.036e+04
Df Residuals:                    3261   BIC:                         1.038e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     